In [17]:
import pandas as pd
import os
import shutil
import glob
from efficientnet_pytorch import EfficientNet

In [18]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu' # device 배정
torch.manual_seed(42)
if device == 'cuda':
    torch.cuda.manual_seed_all(42)
device

'cuda'

In [19]:
torch.cuda.memory_allocated()

21426176

In [20]:
%cd C:\Users\dhkim\Desktop\directory\k-ium\data\2023_k_ium_composition

C:\Users\dhkim\Desktop\directory\k-ium\data\2023_k_ium_composition


In [21]:
train_csv = pd.read_csv('train_set/train.csv')

In [22]:
train_csv

,Index,Aneurysm,L_ICA,R_ICA,L_PCOM,R_PCOM,L_AntChor,R_AntChor,L_ACA,R_ACA,...,R_MCA,L_VA,R_VA,L_PICA,R_PICA,L_SCA,R_SCA,BA,L_PCA,R_PCA
0,1001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1002,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1004,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1005,1,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
4,1006,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,2607,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1123,2608,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1124,2609,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1125,2610,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
pat_number = train_csv['Index'].tolist()
pat_number

[1001,
 1002,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1024,
 1025,
 1026,
 1027,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033,
 1034,
 1035,
 1036,
 1037,
 1038,
 1039,
 1040,
 1041,
 1042,
 1043,
 1044,
 1045,
 1046,
 1047,
 1048,
 1049,
 1050,
 1051,
 1052,
 1053,
 1054,
 1055,
 1056,
 1057,
 1058,
 1059,
 1060,
 1061,
 1062,
 1063,
 1064,
 1065,
 1066,
 1067,
 1068,
 1069,
 1070,
 1071,
 1072,
 1073,
 1074,
 1075,
 1076,
 1077,
 1078,
 1079,
 1080,
 1081,
 1082,
 1083,
 1084,
 1085,
 1086,
 1087,
 1088,
 1089,
 1090,
 1091,
 1092,
 1093,
 1094,
 1095,
 1096,
 1097,
 1098,
 1099,
 1100,
 1101,
 1102,
 1103,
 1104,
 1105,
 1106,
 1107,
 1108,
 1109,
 1110,
 1111,
 1114,
 1115,
 1116,
 1117,
 1118,
 1119,
 1120,
 1121,
 1122,
 1123,
 1124,
 1125,
 1126,
 1127,
 1128,
 1129,
 1130,
 1131,
 1132,
 1133,
 1134,
 1135,
 1136,
 1138,
 1139,
 1140,
 1142,
 1143,
 1144,
 1145,
 1146,
 1148,
 1149,
 1150,

In [24]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2 # albumentations 텐서화 함수

In [25]:

train_transforms = A.Compose([
    A.Resize(224,224),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
#     A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.5),
#     A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
#     A.ChannelShuffle(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # 이미지넷 데이터셋 통계값으로 Normalize
#     A.CoarseDropout(p=0.5),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(224,224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0), # 텐서타입은 안해줌
    ToTensorV2() # Normalize를 먼저하고 tensor화를 진행해야한다.
])

In [26]:
from torch.utils.data import Dataset, DataLoader
import cv2
import os 

class Custom_dataset(Dataset):
    def __init__(self, root_path, mode, transforms):
        self.all_data = sorted(glob.glob(os.path.join(root_path, mode, '*')))
        self.transforms = transforms
        
    def __getitem__(self, index):
        images = []
        if torch.is_tensor(index):
            index = index.tolist()
            
        data_path = self.all_data[index]
        in_data = sorted(glob.glob(os.path.join(data_path, '*')))
        for i in range(8):
            image = cv2.imread(in_data[i])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
            #transform 적용
            if self.transforms is not None:
                augmentation = self.transforms(image = image)
                image = augmentation['image']
            
            images.append(image)
            
            for i in pat_number:
                if str(i) in data_path:
                    patient_index=train_csv[train_csv['Index'] == i]
                    patient_aneurysm = patient_index['Aneurysm']
                    label = int(patient_aneurysm.values)
        
        return images, label       
            
    def __len__(self):
        length = len(self.all_data)
        return length
        
        

In [27]:
root_path = 'C:/Users/dhkim/Desktop/directory/k-ium/data/'

train_class = Custom_dataset(root_path=root_path, mode='train', transforms=train_transforms)
valid_class = Custom_dataset(root_path=root_path, mode='val', transforms=test_transforms)
test_class = Custom_dataset(root_path=root_path, mode='test', transforms=test_transforms)


### Pytorch BatchLoader 생성 (학습에 이용할 최종 dataloader)
from torch.utils.data import DataLoader as DataLoader

batch_size=32

train_loader = DataLoader(train_class, batch_size=batch_size, shuffle = False, num_workers=0)
valid_loader = DataLoader(valid_class, batch_size=batch_size, shuffle = False, num_workers=0)
test_loader = DataLoader(test_class, batch_size=batch_size, shuffle = False, num_workers=0)

In [28]:
from torchvision import models # 모델 라이브러리 함수

# efficientnet_b0 = EfficientNet.from_pretrained('efficientnet-b0').to(device)
efficientnet_b0 = EfficientNet.from_name('efficientnet-b0').to(device)

# finetuning


import torch.nn as nn # 파이토치 뉴럴네트워크 layer 라이브러리
# 입력 이미지의 채널 수를 조정하는 conv1 레이어 수정

efficientnet_b0._conv_stem = nn.Conv2d(24, 32, kernel_size=3, stride=2, padding=1, bias=False).to(device) 

efficientnet_b0._fc = nn.Linear(efficientnet_b0._fc.in_features, 2).to(device) #마지막의 fc 레이어를 바꿈

In [29]:
from torchsummary import summary # 모델 아키텍쳐 확인하는 함수

summary(efficientnet_b0, input_size = (24, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]           6,912
       BatchNorm2d-2         [-1, 32, 112, 112]              64
MemoryEfficientSwish-3         [-1, 32, 112, 112]               0
         ZeroPad2d-4         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-5         [-1, 32, 112, 112]             288
       BatchNorm2d-6         [-1, 32, 112, 112]              64
MemoryEfficientSwish-7         [-1, 32, 112, 112]               0
          Identity-8             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-9              [-1, 8, 1, 1]             264
MemoryEfficientSwish-10              [-1, 8, 1, 1]               0
         Identity-11              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-12             [-1, 32, 1, 1]             288
         Identity-13         [-1, 32, 112, 112]               0
Conv2dStaticSame

In [30]:
lr = 0.000005

optimizer = torch.optim.Adam(efficientnet_b0.parameters(), lr = lr, weight_decay = 1e-8)

In [31]:
from tqdm import tqdm
from torch import optim
import numpy as np
from sklearn.metrics import roc_auc_score


epochs = 30
criterion = nn.CrossEntropyLoss().to(device) # cost function

train_acc_lst, train_loss_lst, test_acc_lst, test_loss_lst= [], [], [], []
state={}
train_auroc_lst= []
test_auroc_lst = []
y_pred = 0
y_true = 0

# 에포크 : training + evaluation
for epoch in range(epochs):
    
    train_loss = 0.0
    total = 0
    correct = 0
    train_acc = 0
    
#     # ----------------- TRAINING  -------------------- 
#     # training 모델로 설정
    efficientnet_b0.train()
    for i, (train_imgs, train_labels) in tqdm(enumerate(train_loader)):
        # gpu에 할당
#         print(len(train_img))
#         print(len(train_label))
        
#         print(len(train_imgs))  # 이미지 리스트의 길이 출력
        
        
        y_true = train_labels.numpy()
        # 이미지 텐서 생성
        train_images = torch.cat((train_imgs), dim=1)
        train_labels = torch.tensor(train_labels)
        

        
        
#         print(train_images.shape)
        
#        24차원됨
        
        
        
        train_img = train_images.to(device)
        train_label = train_labels.to(device)
        
#         efficientnet_b0
        
        output = efficientnet_b0(train_img) # 모델에 입력
        optimizer.zero_grad(set_to_none = True ) # 계산했던 가중치 초기화               
        loss = criterion(output, train_label)
        loss.backward() # 미분
        optimizer.step() # 학습
        
        train_loss += loss.item()
        _, predictions = torch.max(output.data ,dim = 1) 
        y_pred = predictions.cpu().numpy()
        
        total += train_label.size(0)
        correct += (predictions == train_label).sum().item()
        train_acc += 100 * (correct / total)
    
    train_loss = round(train_loss/(i+1), 3) # 소수점 반올림
    train_acc = round(train_acc/(i+1), 3)
    auroc = roc_auc_score(y_true, y_pred)
    print(f'Trainset {epoch}/{epochs} Loss : {train_loss}, Accuracy : {train_acc}%, AUROC : {auroc}')
    train_acc_lst.append(train_acc)
    train_loss_lst.append(train_loss)
    train_auroc_lst.append(auroc)
    
    
  # -------------------------------------------------------------------------------------
    test_loss = 0.0
    corrects = 0
    totals = 0
    test_acc = 0
    
    
#     print("~~~~~~~~~~~~~~~~~~~~~~~~")
    efficientnet_b0.eval()
    for i, (valid_img, valid_label) in enumerate(valid_loader):
        # gpu에 할당
        
        y_true = valid_label.numpy()
        valid_img = torch.cat((valid_img), dim=1)
        valid_label = torch.tensor(valid_label)
        
        valid_img = valid_img.to(device)
        valid_label = valid_label.to(device)
        
        outputs = efficientnet_b0(valid_img) # 모델에 입력
        losses = criterion(outputs, valid_label)
        
         # loss & acc
        test_loss += losses.item()
        _, predictions = torch.max(outputs.data ,dim = 1 )
        y_pred = predictions.cpu().numpy()
        
        
        totals += valid_label.size(0)
        corrects += (predictions == valid_label).sum().item()
        test_acc += 100 * (corrects / totals)
        
    test_loss = round(test_loss/(i+1), 3) # 소수점 반올림
    test_acc = round(test_acc/(i+1), 3)
    auroc = roc_auc_score(y_true, y_pred)
    print(f'Validset {epoch}/{epochs} Loss : {test_loss}, Accuracy : {test_acc}% \n, AUROC : {auroc}')
    test_loss_lst.append(test_loss)
    test_acc_lst.append(test_acc)
    test_auroc_lst.append(auroc)
    
    
    if np.max(test_acc_lst) <= test_acc:
        state['epoch'] = epoch
        state['net'] = efficientnet_b0.state_dict()

        state['train_loss'] = train_loss
        state['test_loss'] = test_loss

        state['train_acc'] = train_acc
        state['test_acc'] = test_acc
# torch.save(state, '/content/drive/MyDrive/Colab Notebooks/dna/week6/efficientnet_b0{}_{}.pth'.format(str(state['epoch']), str(state['test_acc'])))
    
    
    
    
             


0it [00:00, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\1447404477.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(patient_aneurysm.values)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\189222305.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
19it [01:08,  3.62s/it]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\1447404477.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = 

Trainset 0/30 Loss : 0.698, Accuracy : 53.348%, AUROC : 0.5


C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\189222305.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  valid_label = torch.tensor(valid_label)


Validset 0/30 Loss : 0.694, Accuracy : 44.576% 
, AUROC : 0.5


0it [00:00, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\1447404477.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(patient_aneurysm.values)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\189222305.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
19it [01:02,  3.31s/it]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\1447404477.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = 

...

Validset 28/30 Loss : 0.689, Accuracy : 51.346% 
, AUROC : 0.5


0it [00:00, ?it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\1447404477.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(patient_aneurysm.values)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\189222305.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
19it [01:10,  3.71s/it]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\1447404477.py:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = 

Trainset 29/30 Loss : 0.681, Accuracy : 56.229%, AUROC : 0.5571428571428572


C:\Users\Administrator\AppData\Local\Temp\ipykernel_17048\189222305.py:92: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  valid_label = torch.tensor(valid_label)


Validset 29/30 Loss : 0.694, Accuracy : 46.771% 
, AUROC : 0.5
